In [1]:
# nobody knows anything
#                  -Kai
#
# github.com/kaibrooks

In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.layers import Dropout # dropout crew 4 lyf
import tensorflow as tf
from tensorflow import keras as keras
import numpy as np
import random
import sys
import io
import os
import h5py
import matplotlib.pyplot as plt # fancy plots
from matplotlib import pyplot as plt
from datetime import datetime # so we know how much time we've wasted

print('TensorFlow version', tf.__version__)
print('Keras version', keras.__version__)
#print('SciPy version', scipy.__version__)
print('Numpy version', np.__version__) 
#print('Pillow version', PIL.__version__) 

Using TensorFlow backend.


TensorFlow version 2.0.0
Keras version 2.2.4-tf
Numpy version 1.17.4


In [3]:
# doin thangs
epochs=1000 #60
epochOutput = False
temperature = [0.2, 0.5, 1.0, 1.2]
earlyStop = True


np.seterr(divide='ignore') # ignore divide by zero warnings

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
## get text file and parse it

#path = get_file( # use this for http request
#    'afile.txt',
#    origin='http://omega.lul/afile.txt')

# look for a .txt file in /data
source = 'alice'

path = os.path.join('data', source + '.txt')

print('Loaded',path)

# print sweet ~data~ about the file
with open(path, encoding='utf-8', errors='ignore') as f: # errors=ignore strips non utf-8 chars
    text = f.read().lower()

chars = sorted(list(set(text)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('corpus length :', len(text))
print('unique chars  :', len(chars))
print('total patterns:', len(sentences))
print('') # empty line

print('Vectorizing...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('Done')

Loaded data/alice.txt
corpus length : 144343
unique chars  : 44
total patterns: 48101

Vectorizing...
Done


In [5]:
## model
# this has NO TEST DATASET

print('Building model...')
#model = Sequential()
#model.add(LSTM(128, input_shape=(maxlen, len(chars))))
#model.add(Dense(len(chars), activation='softmax'))
#optimizer = RMSprop(learning_rate=0.01)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dropout(0.1))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer) # metrics=['accuracy']

print('Done')

Building model...
Done


In [ ]:
# train me
now = datetime.now()
runDate = now.strftime("%Y-%m-%d_%H-%M-%S")
print("Starting training at", runDate, 'UTC') # lets see how long this takes

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    if epochOutput:
        # print text at each epoch
        print()
        print('**************************** Generating text after Epoch: %d ****************************' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for temperature in [temperature]:
            print('--------------- temperature: ', temperature, '---------------')

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('--------------- seed: <', sentence,'>')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = indices_char[next_index]

                sentence = sentence[1:] + next_char
                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


# early stopping
# loss, accuracy

if earlyStop:
    #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    #es = EarlyStopping(monitor='loss', mode='min', min_delta=1, baseline=0.1)
    es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=round(epochs*1.1)) # stop early if no progress for 10% of max epochs


mc1 = ModelCheckpoint('trained_models/best_model.h5', monitor='loss', mode='min', save_best_only=True)
mc2 = ModelCheckpoint('trained_models/last_model.h5') # save model each run

history = model.fit(x, y,
          batch_size=128,
          epochs=epochs,
          callbacks=[print_callback, es, mc1, mc2]) # add back es!

then = datetime.now()
print("Finished training at", then.strftime("%d-%b-%Y %H:%M:%S")) # lets see how long this took

print('Model summary:')
model.summary()

Starting training at 2019-12-12_06-05-06 UTC
Epoch 1/1000
44416/48101 [==========================>...] - ETA: 4s - loss: 2.4382

In [ ]:
## save models and log

# this is where you pay attention to where you mapped the Docker dir to your local dir at run
save_dir = os.path.join(os.getcwd(), 'trained_models')

if not os.path.isdir(save_dir): # make dir if it doesn't exist
    os.makedirs(save_dir)    
    print(save_dir, 'doesn\'t exist, creating it')

print('Saving...') # do this because saving the model takes 5+ seconds sometimes


# name the files
last_name =  source + '_' + runDate + '_trainedmodel_last.h5'
summary_name = source + '_' + runDate + '_modelsummary.txt'
best_name = source + '_' + runDate + '_trainedmodel_best.h5'


# get the best model
saved_model = load_model('trained_models/best_model.h5')

# save best model
model_path = os.path.join(save_dir, best_name)
saved_model.save(model_path)

# get the last model
last_model = model

# save last model
model_path = os.path.join(save_dir, last_name)
last_model.save(model_path)

# save summary
with open(os.path.join(save_dir, summary_name),'w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))


print('Saved last model at %s ' % os.path.join(save_dir, last_name)) 
print('Saved best model at %s ' % os.path.join(save_dir, best_name)) 
print('Saved summary at %s' % os.path.join(save_dir, summary_name))   

size = os.path.getsize(model_path) # note how big the model is because they're often huge
print('Model is',round(size/1e+6),'MB')

In [ ]:
## plot training information
# accuracy
#plt.plot(history.history['accuracy']) # kx or similar Matlab commands for plotting
#plt.plot(history.history['val_accuracy'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='lower right')
#plt.show()

# loss
loss_min = np.min(history.history['loss'])

plt.plot(history.history['loss'])
#plt.scatter(loss_min, list(range(1, 200)), 'kx')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.savefig(source + '_' + runDate + '_loss.png')


plt.show()

print('Min:', str(loss_min))

In [ ]:
## load a model for testing

#model_path = os.path.join(save_dir, 'last_model.h5') # external file
#model_path = os.path.join(save_dir, last_name) # last
model_path = os.path.join(save_dir, best_name) # best

print('Loaded', model_path)


In [ ]:
## set for the final output
start_index = random.randint(0, len(text) - maxlen - 1)
sentence = text[start_index: start_index + maxlen]
randomizeOut = False
temperature = 1 # 0.5? depends on training length
maxChars = 400 # 400 is same as above

if randomizeOut:
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence

seed = sentence.replace('\n','')
seed = seed.replace('\t','')
print('*** seed: <', seed ,'>')
#sys.stdout.write(generated)

print('\n',seed, end = '')

for i in range(maxChars):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = indices_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()